За основу взят код с гитхаба
https://github.com/sl-iwi/auto_offers_parser/blob/master/app/auto_ru_parser.py  

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [21]:
from IPython.display import display
pd.options.display.max_columns = None

Посмотрим на тестовую выборку:

In [ ]:
test = pd.read_csv('test.csv')

In [22]:
test.sample(10)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,id
19368,седан,INFINITI,синий,бензин,2006.0,G35 3.5 AT (315 л.с.) 4WD,4.0,2007.0,SEDAN AUTOMATIC 3.5,автоматическая,3.5 LTR,315 N12,Продаю машину в отличном состоянии\nТехнически...,176000.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",полный,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,NaN,19368
23512,внедорожник 5 дв.,VOLVO,красный,дизель,2013.0,2.4d AT (181 л.с.) 4WD,5.0,2014.0,ALLROAD_5_DOORS AUTOMATIC 2.4,автоматическая,2.4 LTR,181 N12,✪ ✪ ✪ ✪ ✪ ✪ ✪ ✪ ✪ ✪ ✪ ✪ ✪ ✪ ✪ ✪ ...,138265.0,"['[{""name"":""Комфорт"",""values"":[""Электростеклоп...",полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,23512
24303,лифтбек,SKODA,серебристый,бензин,2012.0,1.6 MT (105 л.с.),5.0,2015.0,LIFTBACK MECHANICAL 1.6,механическая,1.6 LTR,105 N12,Покупал машину у стариков на 9 тыс жизни . Чех...,40000.0,"['[{""name"":""Прочее"",""values"":[""Защита картера""...",передний,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,NaN,24303
29358,седан,HONDA,белый,бензин,2007.0,Type S 2.4 AT (201 л.с.),4.0,2008.0,SEDAN AUTOMATIC 2.4,автоматическая,2.4 LTR,201 N12,Аккуратная эксплуатация. Идеальное состояние....,130000.0,"['[{""name"":""Прочее"",""values"":[""Защита картера""...",передний,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,10 лет,29358
34931,внедорожник 5 дв.,AUDI,чёрный,дизель,2005.0,3.0d AT (233 л.с.) 4WD,5.0,2007.0,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,3.0 LTR,233 N12,"Состояние хорошее, все работает. Двигатель, ко...",344000.0,"['[{""name"":""Обзор"",""values"":[""Противотуманные ...",полный,Левый,Не требует ремонта,3 или более,Дубликат,Растаможен,NaN,34931
7712,внедорожник 5 дв.,NISSAN,чёрный,бензин,2010.0,1.6 CVT (117 л.с.),5.0,2013.0,ALLROAD_5_DOORS VARIATOR 1.6,вариатор,1.6 LTR,117 N12,Прочее: 2. Комфорт: 10. Салон: 11. Безопасност...,45000.0,"['[{""name"":""Прочее"",""values"":[""Защита картера""...",передний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,6 лет и 5 месяцев,7712
356,внедорожник 5 дв.,NISSAN,серый,бензин,2007.0,2.5 CVT (169 л.с.) 4WD,5.0,2008.0,ALLROAD_5_DOORS VARIATOR 2.5,вариатор,2.5 LTR,169 N12,Артикул: 24813\n\nПРИ ПОКУПКЕ АВТОМОБИЛЯ В КРЕ...,117000.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",полный,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,NaN,356
1077,внедорожник 5 дв.,MITSUBISHI,белый,бензин,2012.0,1.8 CVT (140 л.с.),5.0,2013.0,ALLROAD_5_DOORS VARIATOR 1.8,вариатор,1.8 LTR,140 N12,100% история обслуживания: имеются все докумен...,130000.0,"['[{""name"":""Прочее"",""values"":[""Защита картера""...",передний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,6 лет и 4 месяца,1077
17314,седан,HONDA,коричневый,бензин,2005.0,1.8 MT (140 л.с.),4.0,2007.0,SEDAN MECHANICAL 1.8,механическая,1.8 LTR,140 N12,"Надёжный авто, японская сборка, ПТС оригинал, ...",210000.0,[],передний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,NaN,17314
32954,купе,NISSAN,чёрный,бензин,2005.0,3.5 MT (300 л.с.),2.0,2006.0,COUPE MECHANICAL 3.5,механическая,3.5 LTR,300 N12,"Дилерский авто в хорошем, полностью стоковом т...",105000.0,"['[{""name"":""Комфорт"",""values"":[""Круиз-контроль...",задний,Левый,Не требует ремонта,3 или более,Оригинал,Растаможен,6 лет и 9 месяцев,32954


In [5]:
test['brand'].value_counts()

VOLKSWAGEN    3872
AUDI          3837
BMW           3837
TOYOTA        3706
MERCEDES      3671
SKODA         3651
NISSAN        3602
MITSUBISHI    2161
VOLVO         2140
HONDA         1692
LEXUS         1427
INFINITI      1336
SUZUKI        1058
Name: brand, dtype: int64

In [18]:
test.shape

(35990, 23)

In [20]:
test.columns

Index(['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement', 'enginePower',
       'description', 'mileage', 'Комплектация', 'Привод', 'Руль', 'Состояние',
       'Владельцы', 'ПТС', 'Таможня', 'Владение', 'id'],
      dtype='object')

Тестовая выборка состоит из двух блоков информации для парсинга:

Часть bodytype - mileage парсим в первую очередь

Во вторую очередь парсим следующие поля:

'description'
'Комплектация'
'Привод'
'Руль'
'Состояние'
'Владельцы'
'ПТС'
'Таможня'
'Владение'
'id'


### Парсинг:

In [12]:
vendors_list = ['volkswagen', 'audi', 'bmw','toyota','mercedes','skoda','nissan','mitsubishi','volvo','honda','lexus','infiniti','suzuki']

In [8]:
def search_url(vendor):
    location = 'moskva'
    model = 'all'
    url = f"https://auto.ru/{location}/cars/{vendor}/{model}"
    return url


In [9]:
#Функция парсит данные с нескольких страниц
#%%timeit
# this func returns dataframe with offers.
list_autos = []
def get_autoru_offers_all(u):

#    autos = pd.DataFrame(list_autos)
    
    response = requests.get(u)
    soup = BeautifulSoup(response.content.decode('utf-8'), "html.parser")

    module_list = soup.find("div", class_= "ListingCars-module__container ListingCars-module__list")
    children = module_list.findChildren("div", recursive=False)
    for child in children:
        metas = child.find_all("meta")
        offer = {}
        for meta in metas:
            key = meta.get('itemprop')
            value = meta.get('content')
            offer.update({key: value}) #parse parameters except mileage
                
        miles = child.find("div", {'class': 'ListingItem-module__kmAge'})
        offer.update({'mileage': miles}) #parse mileage
        list_autos.append(offer)

    next_page = soup.find('link', attrs={'rel': 'next'})
    if next_page is not None:
#           # If there is "NEXT" button => continue parsing to the next page
        time.sleep(2)
        next_url = next_page['href']
        get_autoru_offers_all(next_url)
    
    autos = pd.DataFrame(list_autos)
    autos = autos.drop(['image', 'availability', 'priceCurrency', 'url'], 1)
    autos.dropna(axis=0, inplace=True) #drop rows with missed values
    
    return autos

Парсинг одной страницы занимает порядка 2-3 сек

Цикл по маркам авто не отрабатывает - слишком много ресурсов - парсила по каждой марке отдельно

In [10]:
#vw
list_autos = []
u = search_url('volkswagen')
train_vw = get_autoru_offers_all(u)
train_vw.shape

(3663, 14)

In [19]:
train_vw.sample()

,mileage,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,price,engineDisplacement,enginePower
843,[102 647 км],компактвэн,VOLKSWAGEN,синий,бензин,2010,1.2 MT,4,2013,COMPACTVAN MECHANICAL 1.2,механическая,555000,1.2 LTR,86 N12


In [16]:
#audi
list_autos = []
u = search_url('audi')
train_audi = get_autoru_offers_all(u)
train_audi.shape

(3663, 14)

In [18]:
train_audi.sample()

,mileage,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,price,engineDisplacement,enginePower
1188,[19 539 км],седан,AUDI,серый,бензин,2017,3.0 AT,4,2018,SEDAN AUTOMATIC 3.0,автоматическая,4820000,3.0 LTR,340 N12


In [47]:
#bmw
list_autos = []
u = search_url('bmw')
train_bmw = get_autoru_offers_all(u)
train_bmw.shape

(3663, 14)

In [48]:
train_bmw.sample()

,mileage,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,price,engineDisplacement,enginePower
3886,[Новый],внедорожник 5 дв.,BMW,синий,дизель,2019,2.0 AT,5,2020,ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,2940900,2.0 LTR,150 N12


In [ ]:
#toyota
list_autos = []
u = search_url('toyota')
train_toyota = get_autoru_offers_all(u)
train_toyota.shape

In [ ]:
train_toyota.sample()

In [ ]:
#mercedes
list_autos = []
u = search_url('mercedes')
train_merc = get_autoru_offers_all(u)
train_merc.shape

In [ ]:
train_merc.sample()

In [ ]:
#skoda
list_autos = []
u = search_url('skoda')
train_skoda = get_autoru_offers_all(u)
train_skoda.shape

In [ ]:
train_skoda.sample()

In [ ]:
#nissan
list_autos = []
u = search_url('nissan')
train_nissan = get_autoru_offers_all(u)
train_nissan.shape

In [ ]:
train_nissan.sample()

In [20]:
#'mitsubishi'
list_autos = []
u = search_url('mitsubishi')
train_mits = get_autoru_offers_all(u)
train_mits.shape

(3663, 14)

In [22]:
train_mits.sample()

,mileage,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,price,engineDisplacement,enginePower
3390,[150 000 км],универсал 5 дв.,MITSUBISHI,серебристый,бензин,2005,1.6 MT,5,2007,WAGON_5_DOORS MECHANICAL 1.6,механическая,400000,1.6 LTR,98 N12


In [23]:
#volvo
list_autos = []
u = search_url('volvo')
train_volvo = get_autoru_offers_all(u)
train_volvo.shape

(2484, 14)

In [25]:
train_volvo.sample()

,mileage,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,price,engineDisplacement,enginePower
1145,[Новый],универсал 5 дв.,VOLVO,белый,бензин,2018,2.0 AT,5,2020,WAGON_5_DOORS AUTOMATIC 2.0,автоматическая,3325600,2.0 LTR,250 N12


In [26]:
#honda
list_autos = []
u = search_url('honda')
train_honda = get_autoru_offers_all(u)
train_honda.shape

(1779, 14)

In [27]:
train_honda.sample()

,mileage,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,price,engineDisplacement,enginePower
822,[125 839 км],внедорожник 5 дв.,HONDA,коричневый,бензин,2012,2.4 AT,5,2014,ALLROAD_5_DOORS AUTOMATIC 2.4,автоматическая,1315000,2.4 LTR,190 N12


In [28]:
#lexus
list_autos = []
u = search_url('lexus')
train_lexus = get_autoru_offers_all(u)
train_lexus.shape

(1748, 14)

In [30]:
train_lexus.sample()

,mileage,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,price,engineDisplacement,enginePower
895,[250 000 км],внедорожник 5 дв.,LEXUS,белый,бензин,2009,4.6 AT,5,2012,ALLROAD_5_DOORS AUTOMATIC 4.6,автоматическая,1850000,4.6 LTR,296 N12


In [32]:
#infiniti
list_autos = []
u = search_url('infiniti')
train_inf = get_autoru_offers_all(u)
train_inf.shape

(1687, 14)

In [33]:
train_inf.sample()

,mileage,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,price,engineDisplacement,enginePower
358,[Новый],внедорожник 5 дв.,INFINITI,чёрный,бензин,2016,3.5 CVT,5,2019,ALLROAD_5_DOORS VARIATOR 3.5,вариатор,3420000,3.5 LTR,283 N12


In [34]:
#suzuki
list_autos = []
u = search_url('suzuki')
train_suz = get_autoru_offers_all(u)
train_suz.shape

(1113, 14)

In [38]:
train_suz.sample()

,mileage,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,price,engineDisplacement,enginePower
848,[151 363 км],хэтчбек 5 дв.,SUZUKI,коричневый,бензин,2013,1.6 CVT,5,2014,HATCHBACK_5_DOORS VARIATOR 1.6,вариатор,549000,1.6 LTR,117 N12


Данные предварительно сохранены в два файла, код далее собирает их вместе и немного обрабатывает столбцы

In [3]:
df1 = pd.read_csv('train_vw_audi_mit_volvo_honda_lexus_inf_suz.csv')
df2 = pd.read_csv('train_bmw_nissan.csv')

In [7]:
df1['brand'].value_counts()

MITSUBISHI    3663
AUDI          3663
VOLKSWAGEN    3663
VOLVO         2484
HONDA         1779
LEXUS         1748
INFINITI      1687
SUZUKI        1113
Name: brand, dtype: int64

In [8]:
df2['brand'].value_counts()

TOYOTA      3663
SKODA       3663
BMW         3663
MERCEDES    3663
NISSAN      3663
Name: brand, dtype: int64

In [10]:
df = pd.concat([df1, df2], axis = 0, join = 'outer', ignore_index = False)
df['brand'].value_counts()

TOYOTA        3663
SKODA         3663
BMW           3663
NISSAN        3663
MITSUBISHI    3663
AUDI          3663
VOLKSWAGEN    3663
MERCEDES      3663
VOLVO         2484
HONDA         1779
LEXUS         1748
INFINITI      1687
SUZUKI        1113
Name: brand, dtype: int64

In [19]:
df.columns

Index(['Unnamed: 0', 'mileage', 'bodyType', 'brand', 'color', 'fuelType',
       'modelDate', 'name', 'numberOfDoors', 'productionDate',
       'vehicleConfiguration', 'vehicleTransmission', 'price',
       'engineDisplacement', 'enginePower'],
      dtype='object')

In [31]:
df_final = df.drop('Unnamed: 0', axis = 1)

In [32]:
df_final.head()

,mileage,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,price,engineDisplacement,enginePower
0,"<div class=""ListingItem-module__kmAge"">42 000 ...",седан,VOLKSWAGEN,чёрный,бензин,2014,1.8 AMT,4,2018,SEDAN ROBOT 1.8,роботизированная,1600000,1.8 LTR,180 N12
1,"<div class=""ListingItem-module__kmAge"">265 800...",универсал 5 дв.,VOLKSWAGEN,серебристый,дизель,2005,2.0 AMT,5,2008,WAGON_5_DOORS ROBOT 2.0,роботизированная,420000,2.0 LTR,140 N12
2,"<div class=""ListingItem-module__kmAge"">236 000...",седан,VOLKSWAGEN,серебристый,бензин,2005,2.0 AT,4,2006,SEDAN AUTOMATIC 2.0,автоматическая,350000,2.0 LTR,200 N12
3,"<div class=""ListingItem-module__kmAge"">230 000...",минивэн,VOLKSWAGEN,серебристый,бензин,2009,2.0 AMT,4,2014,MINIVAN ROBOT 2.0,роботизированная,1050000,2.0 LTR,204 N12
4,"<div class=""ListingItem-module__kmAge"">124 000...",седан,VOLKSWAGEN,чёрный,бензин,2009,1.6 AT,4,2010,SEDAN AUTOMATIC 1.6,автоматическая,385000,1.6 LTR,105 N12


In [33]:
df_final.columns

Index(['mileage', 'bodyType', 'brand', 'color', 'fuelType', 'modelDate',
       'name', 'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'price', 'engineDisplacement', 'enginePower'],
      dtype='object')

In [34]:
cols = ['bodyType', 'brand', 'color', 'fuelType', 'modelDate',
       'name', 'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement', 'enginePower','mileage','price']

In [36]:
df_final = df_final[cols]
df_final.sample(10)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,mileage,price
3313,минивэн,VOLKSWAGEN,зелёный,бензин,2000,1.8 AT,5,2002,MINIVAN AUTOMATIC 1.8,автоматическая,1.8 LTR,150 N12,"<div class=""ListingItem-module__kmAge"">300 000...",320000
10178,седан,MERCEDES,чёрный,бензин,2016,2.0 AT,4,2020,SEDAN AUTOMATIC 2.0,автоматическая,2.0 LTR,197 N12,"<div class=""ListingItem-module__kmAge"">Новый</...",2793200
18790,внедорожник 3 дв.,SUZUKI,серебристый,бензин,2005,1.3 MT,3,2008,ALLROAD_3_DOORS MECHANICAL 1.3,механическая,1.3 LTR,85 N12,"<div class=""ListingItem-module__kmAge"">127 747...",449000
11716,внедорожник 5 дв.,VOLVO,коричневый,дизель,2008,2.4 AT,5,2010,ALLROAD_5_DOORS AUTOMATIC 2.4,автоматическая,2.4 LTR,205 N12,"<div class=""ListingItem-module__kmAge"">164 982...",1050000
4445,внедорожник 5 дв.,AUDI,белый,бензин,2011,2.0 AMT,5,2012,ALLROAD_5_DOORS ROBOT 2.0,роботизированная,2.0 LTR,170 N12,"<div class=""ListingItem-module__kmAge"">145 863...",779980
4592,седан,AUDI,чёрный,бензин,2011,2.0 CVT,4,2012,SEDAN VARIATOR 2.0,вариатор,2.0 LTR,180 N12,"<div class=""ListingItem-module__kmAge"">156 000...",759000
6535,седан,TOYOTA,серый,бензин,2006,1.6 AT,4,2007,SEDAN AUTOMATIC 1.6,автоматическая,1.6 LTR,124 N12,"<div class=""ListingItem-module__kmAge"">160 000...",440000
16082,седан,NISSAN,чёрный,бензин,1994,3.0 MT,4,1996,SEDAN MECHANICAL 3.0,механическая,3.0 LTR,193 N12,"<div class=""ListingItem-module__kmAge"">401 000...",90000
9592,внедорожник 5 дв.,MERCEDES,синий,дизель,2015,3.0 AT,5,2019,ALLROAD_5_DOORS AUTOMATIC 3.0,автоматическая,3.0 LTR,249 N12,"<div class=""ListingItem-module__kmAge"">Новый</...",6251030
9738,внедорожник 5 дв.,MERCEDES,чёрный,бензин,2019,2.0 AT,5,2020,ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,2.0 LTR,249 N12,"<div class=""ListingItem-module__kmAge"">Новый</...",3805920


In [37]:
df_final.to_csv('train_part_one.csv')

In [38]:
DF = pd.read_csv('train_part_one.csv')

In [62]:
DF['mileage'] = DF['mileage'].replace('div',' ')

In [63]:
DF['mileage'][1]

'<div class="ListingItem-module__kmAge">265\xa0800\xa0км</div>'

In [43]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35990 entries, 0 to 35989
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              35990 non-null  object 
 1   brand                 35990 non-null  object 
 2   color                 35990 non-null  object 
 3   fuelType              35990 non-null  object 
 4   modelDate             35990 non-null  float64
 5   name                  35990 non-null  object 
 6   numberOfDoors         35990 non-null  float64
 7   productionDate        35990 non-null  float64
 8   vehicleConfiguration  35990 non-null  object 
 9   vehicleTransmission   35990 non-null  object 
 10  engineDisplacement    35990 non-null  object 
 11  enginePower           35990 non-null  object 
 12  description           35990 non-null  object 
 13  mileage               35990 non-null  float64
 14  Комплектация          35990 non-null  object 
 15  Привод             

In [64]:
DF.shape

(38115, 15)